In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import timeit
from citipy import citipy
from config import weather_api_key
from config import google_map_api_key
from datetime import datetime
import requests
import time
from scipy.stats import linregress
import gmaps
import time

Prepare the list of cities derived from a random list of coordinates

In [2]:
# Create a set of 2,000 latitude/longitude pairs
lats = np.random.uniform(low = -90.000, high=90.000, size=2000)
longs = np.random.uniform(low = -180.000, high=180.000, size=2000)
lats_longs = zip(lats, longs)

In [3]:
# Create a list of latitude/longitude pairs
coords = list(lats_longs)

In [4]:
# Create list of city names using latitude/longitude list created in previous cell
city_list = []

# Loop thru coordinate list and get city name(s)
for coord in coords:
    city = citipy.nearest_city(coord[0], coord[1]).city_name

    # Make sure cities aren't duplicated
    if city not in city_list:
        city_list.append(city)

# for examination and testing, see the length of the city list we get
len(city_list)

738

Prepare for the API call

In [5]:
# Blank list to store data
city_weather_data = []

weather_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

record_counter = 1
set_counter = 1

# Loop thru cities
for i, city in enumerate(city_list):
    # check the counters
    if (i % 50 == 0 and i >= 50):
        # use a timer to avoid exceeding API requests limit
        # adapted from:
        # https://stackoverflow.com/questions/474528/what-is-the-best-way-to-repeatedly-execute-a-function-every-x-seconds

        start_time = time.time()

        time.sleep(180.0 - ((time.time() - start_time) % 180.0))

        set_counter += 1
        record_counter = 1

    city_url = weather_url + "&q=" + city.replace(" ", "+")

    print(f"Processing record {record_counter} | of set {set_counter} | {city}")
    record_counter += 1

    try:
        # Get data from OpenWeatherMap
        weather = requests.get(city_url).json()
        city_lat = weather["coord"]["lat"]                              # latitude
        city_long = weather["coord"]["lon"]                             # longitude
        city_max_temp = weather["main"]["temp_max"]                     # max temperature
        city_humidity = weather["main"]["humidity"]                     # humidity
        city_cloud_percent = weather["clouds"]["all"]                   # percent cloudiness
        city_wind_speed = weather["wind"]["speed"]                      # wind speed
        city_weather_description = weather["weather"][0]["description"] # weather description
        city_country = weather["sys"]["country"]                        # country - not required, but could be useful
        city_date = datetime.utcfromtimestamp(weather["dt"]).strftime('%Y-%m-%d %H:%M:%S') # date - not required, but could be useful
        # add data to list as a dictionary
        city_weather_data.append({"City": city.title(),
                                    "Country": city_country,
                                    "Latitude": city_lat,
                                    "Longitude": city_long,
                                    "Max Temperature": city_max_temp,
                                    "Humidity": city_humidity,
                                    "Percent Clouds": city_cloud_percent,
                                    "Wind Speed": city_wind_speed,
                                    "Description": city_weather_description,
                                    "Date": city_date})
    except:
        print(f"{city} not found, skipping...")
        pass


cessing record 10 | of set 7 | fairbanks
Processing record 11 | of set 7 | prince george
Processing record 12 | of set 7 | kamakwie
Processing record 13 | of set 7 | alice springs
Processing record 14 | of set 7 | juneau
Processing record 15 | of set 7 | kota kinabalu
Processing record 16 | of set 7 | pilar
Processing record 17 | of set 7 | paita
Processing record 18 | of set 7 | esperance
Processing record 19 | of set 7 | morong
Processing record 20 | of set 7 | virginia beach
Processing record 21 | of set 7 | wattegama
Processing record 22 | of set 7 | padang
Processing record 23 | of set 7 | bandiagara
Processing record 24 | of set 7 | gua musang
Processing record 25 | of set 7 | oranjemund
Processing record 26 | of set 7 | alanya
Processing record 27 | of set 7 | mutsamudu
mutsamudu not found, skipping...
Processing record 28 | of set 7 | mahebourg
Processing record 29 | of set 7 | fukue
Processing record 30 | of set 7 | alotau
alotau not found, skipping...
Processing record 31 | o

In [6]:
len(city_weather_data)

675

In [7]:
city_weather_df = pd.DataFrame(city_weather_data)
city_weather_df.head()

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Percent Clouds,Wind Speed,Description,Date
0,Avarua,CK,-21.2078,-159.7750,73.45,73,90,6.91,overcast clouds,2021-06-13 01:47:11
1,Vaini,TO,-21.2000,-175.2000,77.16,69,40,12.66,scattered clouds,2021-06-13 01:47:11
2,Albany,US,42.6001,-73.9662,66.07,82,15,2.30,few clouds,2021-06-13 01:42:32
3,Ahuimanu,US,21.4447,-157.8378,87.51,56,40,13.80,scattered clouds,2021-06-13 01:47:12
4,Iquique,CL,-20.2208,-70.1431,62.64,78,5,3.20,clear sky,2021-06-13 01:47:12


In [8]:
city_weather_df.to_csv("city_weather_csv.csv", index_label="City ID")